In [1]:
import json
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np     
import random

## Import the data
 
We have five JSON files to parse. Each file is a batch of articles and their associated metadata from The Guardian's Content API (CAPI). With a free developer key, we are limited to only 200 articles per request. With a total of five downloads, this brings the total to 1000 on which we can use. If you'd like to query the API yourself, this repo contains an example query in a .txt file. You can sign up for your own developer key at https://open-platform.theguardian.com/ 

The JSON files contain many fields that could be useful for the purposes of machine learning. However, we will focus on the following fields for now:

- trailtext 
- headline
- body (this is the data we will use for machine learning, the above two merely provide context)

We can experiment with the metadata later (such as tags, author etc). Let's initialise our arrays and append with each article's relevant field.

In [2]:
import json

files = ["search_0.json", "search_1.json", "search_2.json", "search_3.json", "search_4.json"]

results = []

for file in files:
        
    # Opening JSON file
    f = open("articles/" + file)
    
    # returns JSON object as 
    # a dictionary
    # print(data)
    data = json.load(f)
    
    # print((data["response"]["results"][0]))
    
    data = data["response"]["results"]
    
    for item in data:
        results.append(data)
        
print (len(results))

1000


In [4]:
files = ["search_0.json", "search_1.json", "search_2.json", "search_3.json", "search_4.json"]

headline = []
trailText = []
body = []

for file in files:
        
    # Opening JSON file
    f = open("articles/" + file)
    
    # returns JSON object as 
    # a dictionary
    data = json.load(f)
        
    #this is the data that we actually care about
    articles = data['response']['results']

    for i in range(len(articles)):
        headline.append((articles[i]["fields"]["headline"]))
        trailText.append((articles[i]["fields"]["trailText"]))
        body.append((articles[i]["fields"]["body"]))



Sanity check the lengths

In [5]:
len(headline), len(trailText), len(body)

(1000, 1000, 1000)

## Preprocessing the data

We should now clean the data as some of the fields contain HTML (which we don't want). 

In [6]:
# body[1]

In [7]:
htmlRemover = re.compile('<.*?>') 
newlineRemover = '\n'

# as per recommendation from @freylis, compile once only

def cleanhtml(raw_html):
    cleantext = re.sub(htmlRemover, '', raw_html)
    cleantext = re.sub(newlineRemover, '', cleantext)
    return cleantext

  

In [8]:
cleanTrailText = []
cleanBody = []

for i in range(len(body)):
    cleanTrailText.append(cleanhtml(trailText[i]))
    cleanBody.append(cleanhtml(body[i]))

In [9]:
# cleanBody[1]

## Transform the articles into a sparse matrix based on similarity

But first ofcourse, we need to perform a tf-idf transformation

In [10]:
vect = TfidfVectorizer(min_df=1, stop_words="english")                                                                                                                                                                                                   
tfidf = vect.fit_transform(cleanBody)                                                                                                                                                                                                                       
pairwise_similarity = tfidf * tfidf.T 

In [11]:
tfidf

<1000x40735 sparse matrix of type '<class 'numpy.float64'>'
	with 339334 stored elements in Compressed Sparse Row format>

In [12]:
#necessary for ensuring we don't just return the same article
arr = pairwise_similarity.toarray()     
np.fill_diagonal(arr, np.nan)     

## Did it work? Let's find out!

Let's draw 10 random articles and find the most similar for each one.

In [14]:
for x in range(1, 10):

    i = random.randint(0,1000)                                                                                                                                                                                                                                                                                                                                                                                                                                  

    input_doc = cleanBody[i]                                                                                                                                                                                                 
    input_idx = cleanBody.index(input_doc)                                                                                                                                                                                                                      
    input_idx                                                                                                                                                                                                                                              
    result_idx = np.nanargmax(arr[input_idx]) 
    print("{}:".format(x))
    print("SAVE FOR LATER ARTICLE: {}".format(headline[i]))
    print("{}".format(trailText[i]))

    print("\n")
    # print("cleanBody[result_idx]   

    print("RECOMMENDED ARTICLE: {}".format(headline[result_idx]))
    print("{}".format(trailText[result_idx]))
    similarity_score = round(arr[input_idx,result_idx], 3)
    print("Similarity score: {}".format(similarity_score))
    print("___________")

1:
SAVE FOR LATER ARTICLE: Written in the Body review – subtle meditation on the sense of touch
Choreographer Charlotte Spencer probes our experience of the world via our bodies, skin and hands


RECOMMENDED ARTICLE: America will never get better if we refuse to diagnose and treat its problems
As we endure shooting after shooting, the US starts to seem like a body being attacked from within by some strange and debilitating illness
Similarity score: 0.082
___________
2:
SAVE FOR LATER ARTICLE: Karolina Pliskova criticises ‘unfair’ WTA Wimbledon ranking points withdrawal
Karolina Pliskova, who reached last year’s Wimbledon final, has said the WTA’s withdrawal of ranking points is ‘a bad decision’


RECOMMENDED ARTICLE: Emma Raducanu eager for Wimbledon return despite lack of ranking points
Wimbledon has been stripped of ranking points over its ban on Russian and Belarusian players, but Raducanu is ‘still looking forward’ to the tournament
Similarity score: 0.502
___________
3:
SAVE FOR L